In [4]:
import $ivy.`org.apache.spark::spark-sql:3.0.0`
import $ivy.`org.plotly-scala::plotly-almond:0.7.6`
import $ivy.`org.postgresql:postgresql:42.2.5`
import $ivy.`net.postgis:postgis-jdbc:2.2.1`
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

Downloaded https://repo1.maven.org/maven2/net/postgis/postgis-jdbc/2.2.1/postgis-jdbc-2.2.1.pom
Downloaded https://repo1.maven.org/maven2/net/postgis/postgis-java-aggregator/2.2.1/postgis-java-aggregator-2.2.1.pom
Downloaded https://repo1.maven.org/maven2/org/postgresql/postgresql/maven-metadata.xml
Downloaded https://repo1.maven.org/maven2/org/postgresql/postgresql/42.2.14.jre7/postgresql-42.2.14.jre7.pom
Downloaded https://repo1.maven.org/maven2/net/postgis/postgis-jdbc/2.2.1/postgis-jdbc-2.2.1-sources.jar
Downloaded https://repo1.maven.org/maven2/net/postgis/postgis-jdbc/2.2.1/postgis-jdbc-2.2.1.jar
Downloaded https://repo1.maven.org/maven2/org/postgresql/postgresql/42.2.14.jre7/postgresql-42.2.14.jre7.jar
Downloaded https://repo1.maven.org/maven2/org/postgresql/postgresql/42.2.14.jre7/postgresql-42.2.14.jre7-sources.jar


import $ivy.$                                  

import $ivy.$                                      

import $ivy.$                                 

import $ivy.$                               

import org.apache.log4j.{Level, Logger}

import org.apache.spark.sql._

import org.apache.spark.sql.functions._

In [2]:
implicit class RichDF(val ds:DataFrame) {
    def showHTML(limit:Int = 20, truncate: Int = 20) = {
        import xml.Utility.escape
        val data = ds.take(limit)
        val header = ds.schema.fieldNames.toSeq        
        val rows: Seq[Seq[String]] = data.map { row =>
          row.toSeq.map { cell =>
            val str = cell match {
              case null => "null"
              case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
              case array: Array[_] => array.mkString("[", ", ", "]")
              case seq: Seq[_] => seq.mkString("[", ", ", "]")
              case _ => cell.toString
            }
            if (truncate > 0 && str.length > truncate) {
              // do not show ellipses for strings shorter than 4 characters.
              if (truncate < 4) str.substring(0, truncate)
              else str.substring(0, truncate - 3) + "..."
            } else {
              str
            }
          }: Seq[String]
        }

        publish.html(s""" <table>
                <tr>
                 ${header.map(h => s"<th>${escape(h)}</th>").mkString}
                </tr>
                ${rows.map { row =>
                  s"<tr>${row.map{c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
                }.mkString}
            </table>
        """)        
    }
}

defined class RichDF

In [5]:
val spark = NotebookSparkSession
    .builder
    .master("local")
    .appName("jupyter-scala")
    .getOrCreate

import spark.implicits._

Loading spark-stubs


Downloaded https://repo1.maven.org/maven2/sh/almond/spark-stubs_30_2.12/0.10.1/spark-stubs_30_2.12-0.10.1.pom
Downloaded https://repo1.maven.org/maven2/sh/almond/spark-stubs_30_2.12/0.10.1/spark-stubs_30_2.12-0.10.1-sources.jar
Downloaded https://repo1.maven.org/maven2/sh/almond/spark-stubs_30_2.12/0.10.1/spark-stubs_30_2.12-0.10.1.jar


Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

spark: SparkSession = org.apache.spark.sql.SparkSession@76a88de2
import spark.implicits._

In [11]:
def stackGames(dataType:String, seasonType: String) = {
    val regGames = spark.read
        .option("inferSchema", "true")
        .option("header", "true")
        .csv(s"../$dataType/$seasonType/*.csv")
    
    val teamGameScore = if (dataType == "games_data") {
        val keepCols = regGames.columns.filterNot(x => x.startsWith("away") || x.startsWith("home")).map(col)
        val structCol = 
            explode(array(
                struct(
                    $"home_team" as "team",
                    $"home_score" as "score",
                    lit(1) as "home"
                ),
                struct(
                    $"away_team" as "team",
                    $"away_score" as "score",
                    lit(0) as "home"
                )
            )) as "team_score"
        regGames.select((keepCols :+ structCol): _*)
            .select((keepCols :+ $"team_score.*"): _*)
    } else {
        regGames
    }
    
    teamGameScore.write.mode("overwrite").parquet(s"../$dataType/$seasonType/stackedGames")
    teamGameScore
}

val seasonTypes = Seq("post_season", "pre_season", "regular_season")
val dataTypes = Seq("games_data", "play_by_play_data", "roster_data")

defined function stackGames
seasonTypes: Seq[String] = List("post_season", "pre_season", "regular_season")
dataTypes: Seq[String] = List("games_data", "play_by_play_data", "roster_data")

In [10]:
Seq(1) ++ Seq(2)

res9: Seq[Int] = List(1, 2)

In [13]:
val allFrames = dataTypes.map {
    dt => dt -> seasonTypes.map {
        st => st -> stackGames(dt, st)
    }
}

csv at cmd10.sc:5

1 / 1

csv at cmd10.sc:5

1 / 1

parquet at cmd10.sc:28

...

csv at cmd10.sc:5

1 / 1

parquet at cmd10.sc:28

1 / 1

allFrames: Seq[(String, Seq[(String, DataFrame)])] = List(
  (
    "games_data",
    List(
      ("post_season", [_c0: string, type: string ... 8 more fields]),
      ("pre_season", [type: string, game_id: int ... 7 more fields]),
      ("regular_season", [type: string, game_id: int ... 7 more fields])
    )
  ),
  (
    "play_by_play_data",
    List(
      ("post_season", [play_id: int, game_id: string ... 254 more fields]),
      ("pre_season", [play_id: int, game_id: int ... 254 more fields]),
      ("regular_season", [play_id: int, game_id: int ... 254 more fields])
    )
  ),
  (
    "roster_data",
    List(
      ("post_season", [season: int, season_type: string ... 5 more fields]),
      ("pre_season", [season: int, season_type: string ... 5 more fields]),
      ("regular_season", [season: int, season_type: string ... 5 more fields])
    )
  )
)

In [15]:
val allFramesMap = allFrames.toMap.mapValues(_.toMap)

allFramesMap: Map[String, Map[String, DataFrame]] = Map(
  "games_data" -> Map(
    "post_season" -> [_c0: string, type: string ... 8 more fields],
    "pre_season" -> [type: string, game_id: int ... 7 more fields],
    "regular_season" -> [type: string, game_id: int ... 7 more fields]
  ),
  "play_by_play_data" -> Map(
    "post_season" -> [play_id: int, game_id: string ... 254 more fields],
    "pre_season" -> [play_id: int, game_id: int ... 254 more fields],
    "regular_season" -> [play_id: int, game_id: int ... 254 more fields]
  ),
  "roster_data" -> Map(
    "post_season" -> [season: int, season_type: string ... 5 more fields],
    "pre_season" -> [season: int, season_type: string ... 5 more fields],
    "regular_season" -> [season: int, season_type: string ... 5 more fields]
  )
)

In [18]:
allFramesMap("games_data").mapValues(_.show)

show at cmd17.sc:1

1 / 1

+---+----+----------+----+------+-------------+--------------------+----+-----+----+
|_c0|type|   game_id|week|season|state_of_game|            game_url|team|score|home|
+---+----+----------+----+------+-------------+--------------------+----+-----+----+
|  1|post|2020012600|  18|  2019|          PRE|http://www.nfl.co...| APR|   NA|   1|
|  1|post|2020012600|  18|  2019|          PRE|http://www.nfl.co...| NPR|   NA|   0|
|  2|post|2020010400|  18|  2019|         POST|http://www.nfl.co...| HOU|   22|   1|
|  2|post|2020010400|  18|  2019|         POST|http://www.nfl.co...| BUF|   19|   0|
|  3|post|2020010401|  18|  2019|         POST|http://www.nfl.co...|  NE|   13|   1|
|  3|post|2020010401|  18|  2019|         POST|http://www.nfl.co...| TEN|   20|   0|
|  4|post|2020010500|  18|  2019|         POST|http://www.nfl.co...|  NO|   20|   1|
|  4|post|2020010500|  18|  2019|         POST|http://www.nfl.co...| MIN|   26|   0|
|  5|post|2020010501|  18|  2019|         POST|http://www.nfl.co.

show at cmd17.sc:1

1 / 1

+----+----------+----+------+-------------+--------------------+----+-----+----+
|type|   game_id|week|season|state_of_game|            game_url|team|score|home|
+----+----------+----+------+-------------+--------------------+----+-----+----+
| pre|2015080951|   0|  2015|         POST|http://www.nfl.co...| MIN|   14|   1|
| pre|2015080951|   0|  2015|         POST|http://www.nfl.co...| PIT|    3|   0|
| pre|2015081351|   1|  2015|         POST|http://www.nfl.co...| BAL|   30|   1|
| pre|2015081351|   1|  2015|         POST|http://www.nfl.co...|  NO|   27|   0|
| pre|2015081352|   1|  2015|         POST|http://www.nfl.co...| DET|   23|   1|
| pre|2015081352|   1|  2015|         POST|http://www.nfl.co...| NYJ|    3|   0|
| pre|2015081353|   1|  2015|         POST|http://www.nfl.co...|  NE|   11|   1|
| pre|2015081353|   1|  2015|         POST|http://www.nfl.co...|  GB|   22|   0|
| pre|2015081354|   1|  2015|         POST|http://www.nfl.co...| CLE|   17|   1|
| pre|2015081354|   1|  2015

show at cmd17.sc:1

1 / 1

+----+----------+----+------+-------------+--------------------+----+-----+----+
|type|   game_id|week|season|state_of_game|            game_url|team|score|home|
+----+----------+----+------+-------------+--------------------+----+-----+----+
| reg|2015091000|   1|  2015|         POST|http://www.nfl.co...|  NE|   28|   1|
| reg|2015091000|   1|  2015|         POST|http://www.nfl.co...| PIT|   21|   0|
| reg|2015091300|   1|  2015|         POST|http://www.nfl.co...| CHI|   23|   1|
| reg|2015091300|   1|  2015|         POST|http://www.nfl.co...|  GB|   31|   0|
| reg|2015091306|   1|  2015|         POST|http://www.nfl.co...| HOU|   20|   1|
| reg|2015091306|   1|  2015|         POST|http://www.nfl.co...|  KC|   27|   0|
| reg|2015091305|   1|  2015|         POST|http://www.nfl.co...| NYJ|   31|   1|
| reg|2015091305|   1|  2015|         POST|http://www.nfl.co...| CLE|   10|   0|
| reg|2015091304|   1|  2015|         POST|http://www.nfl.co...| BUF|   27|   1|
| reg|2015091304|   1|  2015

res17: Map[String, Unit] = Map(
  "post_season" -> (),
  "pre_season" -> (),
  "regular_season" -> ()
)

In [7]:
val playByPlay2019 = spark.read.option("header", "true").option("inferSchema", "true").csv("../play_by_play_data/regular_season/*pbp*.csv")

csv at cmd6.sc:1

1 / 1

csv at cmd6.sc:1

11 / 11

playByPlay2019: DataFrame = [play_id: int, game_id: int ... 254 more fields]

In [10]:
val groupThrees = playByPlay2019.columns.groupBy(x => x.slice(0, 3))

groupThrees: Map[String, Array[String]] = Map(
  "pas" -> Array(
    "pass_length",
    "pass_location",
    "pass_attempt",
    "pass_touchdown",
    "passer_player_id",
    "passer_player_name",
    "pass_defense_1_player_id",
    "pass_defense_1_player_name",
    "pass_defense_2_player_id",
    "pass_defense_2_player_name"
  ),
  "des" -> Array("desc"),
  "for" -> Array(
    "forced_fumble_player_1_team",
    "forced_fumble_player_1_player_id",
    "forced_fumble_player_1_player_name",
    "forced_fumble_player_2_team",
    "forced_fumble_player_2_player_id",
    "forced_fumble_player_2_player_name"
  ),
  "sho" -> Array("shotgun"),
  "qua" -> Array("quarter_seconds_remaining", "quarter_end"),
  "tou" -> Array("touchback", "touchdown"),
  "pun" -> Array(
    "punt_blocked",
    "punt_inside_twenty",
    "punt_in_endzone",
    "punt_out_of_bounds",
    "punt_downed",
    "punt_fair_catch",
    "punt_attempt",
    "punt_returner_player_id",
    "punt_returner_player_name",
    "punter

In [14]:
groupThrees("rus")

res13: Array[String] = Array(
  "rush_attempt",
  "rush_touchdown",
  "rusher_player_id",
  "rusher_player_name"
)

In [20]:
val threeStructs = groupThrees.map {
    case (three, cols) => struct(cols.map(col): _*) as three
}
val threeColsNest = playByPlay2019.select(threeStructs.toSeq: _*)

threeStructs: collection.immutable.Iterable[Column] = List(
  struct(pass_length, pass_location, pass_attempt, pass_touchdown, passer_player_id, passer_player_name, pass_defense_1_player_id, pass_defense_1_player_name, pass_defense_2_player_id, pass_defense_2_player_name) AS `pas`,
  struct(desc) AS `des`,
  struct(forced_fumble_player_1_team, forced_fumble_player_1_player_id, forced_fumble_player_1_player_name, forced_fumble_player_2_team, forced_fumble_player_2_player_id, forced_fumble_player_2_player_name) AS `for`,
  struct(shotgun) AS `sho`,
  struct(quarter_seconds_remaining, quarter_end) AS `qua`,
  struct(touchback, touchdown) AS `tou`,
  struct(punt_blocked, punt_inside_twenty, punt_in_endzone, punt_out_of_bounds, punt_downed, punt_fair_catch, punt_attempt, punt_returner_player_id, punt_returner_player_name, punter_player_id, punter_player_name) AS `pun`,
  struct(return_touchdown, return_team, return_yards) AS `ret`,
  struct(first_down_rush, first_down_pass, first_down_penal

In [22]:
threeColsNest.printSchema

root
 |-- pas: struct (nullable = false)
 |    |-- pass_length: string (nullable = true)
 |    |-- pass_location: string (nullable = true)
 |    |-- pass_attempt: string (nullable = true)
 |    |-- pass_touchdown: string (nullable = true)
 |    |-- passer_player_id: string (nullable = true)
 |    |-- passer_player_name: string (nullable = true)
 |    |-- pass_defense_1_player_id: string (nullable = true)
 |    |-- pass_defense_1_player_name: string (nullable = true)
 |    |-- pass_defense_2_player_id: string (nullable = true)
 |    |-- pass_defense_2_player_name: string (nullable = true)
 |-- des: struct (nullable = false)
 |    |-- desc: string (nullable = true)
 |-- for: struct (nullable = false)
 |    |-- forced_fumble_player_1_team: string (nullable = true)
 |    |-- forced_fumble_player_1_player_id: string (nullable = true)
 |    |-- forced_fumble_player_1_player_name: string (nullable = true)
 |    |-- forced_fumble_player_2_team: string (nullable = true)
 |    |-- forced_fumble_

In [30]:
val testCols = playByPlay2019.columns
    .filter(x => x.contains("yds") || x.contains("yards") || x.contains("pass") || x.contains("rush") || x.contains("player"))
    .filterNot(_.contains("lateral"));
playByPlay2019.select(testCols.head, testCols.tail: _*).showHTML()

take at cmd17.sc:4

1 / 1

ydstogo,ydsnet,yards_gained,pass_length,pass_location,air_yards,yards_after_catch,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,total_home_rush_wpa,total_away_rush_wpa,total_home_pass_wpa,total_away_pass_wpa,first_down_rush,first_down_pass,incomplete_pass,rush_attempt,pass_attempt,pass_touchdown,rush_touchdown,complete_pass,passer_player_id,passer_player_name,receiver_player_id,receiver_player_name,rusher_player_id,rusher_player_name,interception_player_id,interception_player_name,punt_returner_player_id,punt_returner_player_name,kickoff_returner_player_name,kickoff_returner_player_id,punter_player_id,punter_player_name,kicker_player_name,kicker_player_id,own_kickoff_recovery_player_id,own_kickoff_recovery_player_name,blocked_player_id,blocked_player_name,tackle_for_loss_1_player_id,tackle_for_loss_1_player_name,tackle_for_loss_2_player_id,tackle_for_loss_2_player_name,qb_hit_1_player_id,qb_hit_1_player_name,qb_hit_2_player_id,qb_hit_2_player_name,forced_fumble_player_1_team,forced_fumble_player_1_player_id,forced_fumble_player_1_player_name,forced_fumble_player_2_team,forced_fumble_player_2_player_id,forced_fumble_player_2_player_name,solo_tackle_1_player_id,solo_tackle_2_player_id,solo_tackle_1_player_name,solo_tackle_2_player_name,assist_tackle_1_player_id,assist_tackle_1_player_name,assist_tackle_2_player_id,assist_tackle_2_player_name,assist_tackle_3_player_id,assist_tackle_3_player_name,assist_tackle_4_player_id,assist_tackle_4_player_name,pass_defense_1_player_id,pass_defense_1_player_name,pass_defense_2_player_id,pass_defense_2_player_name,fumbled_1_player_id,fumbled_1_player_name,fumbled_2_player_id,fumbled_2_player_name,fumble_recovery_1_yards,fumble_recovery_1_player_id,fumble_recovery_1_player_name,fumble_recovery_2_yards,fumble_recovery_2_player_id,fumble_recovery_2_player_name,return_yards,penalty_player_id,penalty_player_name,penalty_yards
0,0,0,NA,NA,NA,NA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,S.Gostkowski,00-0024333,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,0,NA,NA,NA
10,18,18,NA,NA,NA,NA,-1.6933879964717005,1.6933879964717005,0.0,0.0,-0.0526828771195621,0.0526828771195621,0.0,0.0,1,0,0,1,0,0,0,0,NA,NA,NA,NA,00-0024242,De.Williams,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,00-0029669,NA,D.Hightower,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,0,NA,NA,NA
10,31,9,short,right,-4,13,-1.6933879964717005,1.6933879964717005,-1.1286512907955397,1.1286512907955397,-0.0526828771195621,0.0526828771195621,-0.03693286724166811,0.03693286724166811,0,0,0,0,1,0,0,1,00-0022924,B.Roethlisberger,00-0027793,A.Brown,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,00-0029669,NA,D.Hightower,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,0,NA,NA,NA
1,31,4,NA,NA,NA,NA,-1.5655705018557404,1.5655705018557404,-1.1286512907955397,1.1286512907955397,-0.05062911307962792,0.05062911307962792,-0.03693286724166811,0.03693286724166811,1,0,0,1,0,0,0,0,NA,NA,NA,NA,00-0024242,De.Williams,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,00-0030421,J.Collins,00-0032261,M.Brown,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,0,NA,NA,NA
10,45,14,short,right,9,5,-1.5655705018557404,1.5655705018557404,-1.9333188679019382,1.9333188679019382,-0.05062911307962792,0.05062911307962792,-0.0659642312419525,0.0659642312419525,0,1,0,0,1,0,0,1,00-0022924,B.Roethlisberger,00-0023465,H.Miller,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,00-0026150,NA,J.Mayo,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,0,NA,NA,NA
10,56,11,NA,NA,NA,NA,-2.017749761799263,2.017749761799263,-1.9333188679019382,1.9333188679019382,-0.06768794144534318,0.0676879414453431

testCols: Array[String] = Array(
  "ydstogo",
  "ydsnet",
  "yards_gained",
  "pass_length",
  "pass_location",
  "air_yards",
  "yards_after_catch",
  "total_home_rush_epa",
  "total_away_rush_epa",
  "total_home_pass_epa",
  "total_away_pass_epa",
  "total_home_rush_wpa",
  "total_away_rush_wpa",
  "total_home_pass_wpa",
  "total_away_pass_wpa",
  "first_down_rush",
  "first_down_pass",
  "incomplete_pass",
  "rush_attempt",
  "pass_attempt",
  "pass_touchdown",
  "rush_touchdown",
  "complete_pass",
  "passer_player_id",
  "passer_player_name",
  "receiver_player_id",
  "receiver_player_name",
  "rusher_player_id",
  "rusher_player_name",
  "interception_player_id",
  "interception_player_name",
  "punt_returner_player_id",
  "punt_returner_player_name",
  "kickoff_returner_player_name",
  "kickoff_returner_player_id",
  "punter_player_id",
  "punter_player_name",
  "kicker_player_name",
...

In [39]:
val offEpa = playByPlay2019
    .groupBy("posteam")
    .agg(
        countDistinct("game_id") as "n_games",
        sum("epa") as "off_total_epa"
    ).withColumn("off_epa_per_game", $"off_total_epa"/$"n_games")
val defEpa = playByPlay2019
    .groupBy("defteam")
    .agg(
        countDistinct("game_id") as "n_games",
        sum("epa") as "def_total_epa"
    ).withColumn("def_epa_per_game", -$"def_total_epa"/$"n_games")
val teamEpa = offEpa.join(defEpa, $"posteam" === $"defteam").cache

offEpa: DataFrame = [posteam: string, n_games: bigint ... 2 more fields]
defEpa: DataFrame = [defteam: string, n_games: bigint ... 2 more fields]
teamEpa: Dataset[Row] = [posteam: string, n_games: bigint ... 6 more fields]

In [40]:
teamEpa.show

show at cmd39.sc:1

1 / 1

show at cmd39.sc:1

4 / 4

show at cmd39.sc:1

20 / 20

show at cmd39.sc:1

94 / 94

+-------+-------+-------------------+--------------------+-------+-------+-------------------+--------------------+
|posteam|n_games|      off_total_epa|    off_epa_per_game|defteam|n_games|      def_total_epa|    def_epa_per_game|
+-------+-------+-------------------+--------------------+-------+-------+-------------------+--------------------+
|    NYJ|    176| -748.0090033013352|  -4.250051155121223|    NYJ|    176|-250.77222472769537|  1.4248421859528146|
|    CAR|    176|  46.96721348177905| 0.26685916751010824|    CAR|    176|  129.5403939288832| -0.7360249655050182|
|     LA|     64| 214.36348758132652|   3.349429493458227|     LA|     64|  19.96122720983879| -0.3118941751537311|
|     TB|    176| -327.9170150393343| -1.8631648581780356|     TB|    176|  552.9399312801525|  -3.141704155000866|
|    OAK|    176|-309.88359735790453| -1.7607022577153666|    OAK|    176|  920.5027663135909|  -5.230129354054494|
|    DET|    176| 100.54614125066978|  0.5712848934697147|    DET|    17